In [2]:
from collections import Counter

import pandas as pd
import numpy as np

from Code.UtilityFunctions.get_data_path import get_path
from Code.UtilityFunctions.wikidata_functions import categories_dict_singular

In [3]:
biz = pd.read_json(get_path("yelp_academic_dataset_business.json"), lines=True)
categories = list(biz['categories'].str.cat(sep=', ').split(sep=', '))
categories_dict_singular = categories_dict_singular(categories)

category_occurences = pd.DataFrame(list(dict(Counter(categories)).items()),
                                   columns=['category', 'occurences'
                                            ]).sort_values(by='occurences',
                                                           ascending=False)
# Maps the split categories to the original categories
category_occurences['split_category'] = category_occurences['category'].map(categories_dict_singular)
category_occurences = category_occurences.explode('split_category')

# Maps the yelp categories that are already mapped to a schemaType to the original category.
class_mapping = pd.read_csv(get_path('class_mappings.csv'))
category_occurences['split_category'] = category_occurences['split_category'].apply(lambda x: x.title().replace(' ', ''))
category_occurences = category_occurences.merge(class_mapping,
                                                left_on='split_category',
                                                right_on='YelpCategory',
                                                how='left')

In [4]:
category_occurences

,category,occurences,split_category,YelpCategory,SchemaType
0,Restaurants,52268,Restaurant,NaN,NaN
1,Food,27781,Food,NaN,NaN
2,Shopping,24395,Shopping,Shopping,['Retail']
3,Home Services,14356,HomeService,NaN,NaN
4,Beauty & Spas,14292,Beauty,NaN,NaN
...,...,...,...,...,...
1422,Beach Bars,1,BeachBar,NaN,NaN
1423,DUI Schools,1,DuiSchool,NaN,NaN
1424,Patent Law,1,PatentLaw,NaN,NaN
1425,Housing Cooperatives,1,HousingCooperative,NaN,NaN


In [14]:
non_mapped_df = category_occurences[(category_occurences["YelpCategory"].isna()) | (category_occurences["SchemaType"].isna())]
non_mapped_df['category'] = non_mapped_df['category'].str.lower()

C:\Users\madsc\AppData\Local\Temp\ipykernel_2164\427724565.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_mapped_df['category'] = non_mapped_df['category'].str.lower()


,category,occurences,split_category,YelpCategory,SchemaType
0,restaurants,52268,Restaurant,NaN,NaN
1,food,27781,Food,NaN,NaN
3,home services,14356,HomeService,NaN,NaN
4,beauty & spas,14292,Beauty,NaN,NaN
5,beauty & spas,14292,Spa,NaN,NaN
...,...,...,...,...,...
1422,beach bars,1,BeachBar,NaN,NaN
1423,dui schools,1,DuiSchool,NaN,NaN
1424,patent law,1,PatentLaw,NaN,NaN
1425,housing cooperatives,1,HousingCooperative,NaN,NaN


In [ ]:
non_mapped = list(non_mapped_df['category'].unique())
non_mapped

In [2]:
from Code.UtilityFunctions.run_query import run_query

query = """
SELECT DISTINCT ?p
WHERE {
    ?s ?p ?o .
    FILTER regex(?p, "^https://example.org/")
}
"""

predicates = run_query(query, as_dataframe=True)

In [3]:
pd.options.display.max_rows(None)

NameError: name 'pd' is not defined